In [55]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier


In [56]:
# Load the data
file_path = Path('Resources/teds_2019_reduced.csv')
teds_reduced_df = pd.read_csv(file_path)

teds_reduced_df.head()

,CASEID,AGE,RACE,ETHNIC,EDUC,MARSTAT,PRIMINC,PRIMPAY,EMPLOY,EMPLOY_D,DSMCRIT,LOS,REASON,SUCCESSFUL
0,20191553576,4,1,4,4,1,1,7,2,2,19,37,1,1
1,20191465214,5,1,4,3,1,1,7,1,1,4,35,3,0
2,20191443889,1,8,4,2,1,4,4,4,4,19,35,3,0
3,20191409377,5,1,4,3,1,4,7,3,3,4,37,1,1
4,20191479567,11,8,4,3,3,4,7,4,4,4,37,3,0


In [57]:
# Filter all lengths of stay (LOS) > 15 days
LOS25_df = teds_reduced_df[teds_reduced_df['LOS'] > 15]
LOS25_df.head()

,CASEID,AGE,RACE,ETHNIC,EDUC,MARSTAT,PRIMINC,PRIMPAY,EMPLOY,EMPLOY_D,DSMCRIT,LOS,REASON,SUCCESSFUL
0,20191553576,4,1,4,4,1,1,7,2,2,19,37,1,1
1,20191465214,5,1,4,3,1,1,7,1,1,4,35,3,0
2,20191443889,1,8,4,2,1,4,4,4,4,19,35,3,0
3,20191409377,5,1,4,3,1,4,7,3,3,4,37,1,1
4,20191479567,11,8,4,3,3,4,7,4,4,4,37,3,0


In [58]:
count_row = LOS25_df.shape[0] 
print(count_row)

986745


# Create intitial neural network model

In [59]:
# Split our preprocessed data into our features and target arrays

y = LOS25_df["SUCCESSFUL"].values
X = LOS25_df.drop(["REASON","SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\syode\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [60]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [61]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 30)                390       
                                                                 
 dense_16 (Dense)            (None, 10)                310       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 711
Trainable params: 711
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=10)

Epoch 1/10
23127/23127 [==============================] - 36s 2ms/step - loss: 4474153.5000 - accuracy: 0.5721
Epoch 2/10
23127/23127 [==============================] - 32s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 3/10
23127/23127 [==============================] - 32s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 4/10
23127/23127 [==============================] - 34s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 5/10
23127/23127 [==============================] - 33s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 6/10
23127/23127 [==============================] - 32s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 7/10
23127/23127 [==============================] - 32s 1ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 8/10
23127/23127 [==============================] - 35s 2ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 9/10
23127/23127 [==============================] - 35s 2ms/step - loss: 0.6826 - accuracy: 0.5725
Epoch 10/10
23127/23127 [========================

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

7709/7709 - 10s - loss: 0.6821 - accuracy: 0.5743 - 10s/epoch - 1ms/step
Loss: 0.6821427345275879, Accuracy: 0.574343204498291


# Create initial Random Forest Classifier Model

In [65]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.721
